<a href="https://colab.research.google.com/github/JLMR-Code-Creator/Taller_VC_Codigo/blob/main/DeepGA_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Installing Libraries**

In [ ]:
!pip install torch torchvision torchaudio torchtext

In [ ]:
!pip install pytorch-forecasting

In [ ]:
pip install pandas

In [ ]:
pip install xlwt

**Importing Libraries**

In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset
from torch import nn
import matplotlib.pyplot as plt
import numpy as np
import torch.nn.functional as F
from torchsummary import summary
from torch import optim
import torchvision
from torchvision import transforms, utils
from pytorch_forecasting.metrics import SMAPE, RMSE, MAPE
import cv2
import random
import math
import scipy.io

**Loading GPU**

In [ ]:
if torch.cuda.is_available():
  device = torch.device("cuda:0")
else:
  device = torch.device("cpu")
print(device)

cuda:0


**Loading Data**

In [ ]:
#Mounting Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Los datos deben estar en una carpeta llamada "DB" que este en la carpeta raiz de google drive. Dentro de "DB" debe existir una carpeta llamada "MAT" que contenga las 100 diferentes sub-carpetas, llamadas "carpeta_X", donde X es el número de carpeta.

In [ ]:
#Images path
root = '/content/gdrive/My Drive/DB46HSILAB/MAT/carpeta_'
labels_root = '/content/gdrive/My Drive/DB46HSILAB/complementos/clase.csv'

#Reading labels
labels = np.genfromtxt(labels_root, delimiter = ',')[:,4]

In [ ]:
#Dataset Class
class Dataset(Dataset):
  def __init__(self, data_type, labels,transform = None):
    self.root = root #The folder path
    self.labels = labels
    self.transform = transform #Transform composition
    self.data_type = data_type

  def __len__(self):
    return 46

  def __getitem__(self, idx):
    if torch.is_tensor(idx):
      idx = idx.tolist()
    root = self.root[:]

    #Defining roots
    n_root = root + str(idx+1) + '/z.mat'

    #Reading files
    mat = scipy.io.loadmat(n_root)

    if self.data_type == 'train':
        ab = mat['hsi_hs_e']
        la = mat['hsi_hi_e']
        lb = mat['hsi_si_e']

    else:
        ab = mat['hsi_hs_p']
        la = mat['hsi_hi_p']
        lb = mat['hsi_si_p']

    #Concatenating files
    img = np.dstack((ab,la,lb))
    values = mat['valantocianinas']
    anto = np.array(values)
    label = np.mean(anto)
    label = label.round(decimals=2)
    #print(label)
    #Reading label
    #label = self.labels[idx]
    sample = {'image': img, 'label': label}

    if self.transform:
      sample = self.transform(sample)

    return sample

#Class to transform image to tensor
class ToTensor(object):
  def __call__(self, sample):
    image, label = sample['image'], sample['label']

    #Swap dimmensions because:
    #       numpy image: H x W x C
    #       torch image: C x H x W
    #print(image.shape)
    image = image.transpose((2,0,1))
    #print(image.shape)
    return {'image':torch.from_numpy(image),
            'label':label}

In [ ]:
#Loading Datasets
train_ds = Dataset(data_type = 'train', labels = labels, transform = transforms.Compose([ToTensor()]))
test_ds = Dataset(data_type = 'test', labels = labels, transform = transforms.Compose([ToTensor()]))

In [ ]:
#Testing
i = 0
print("Length of Training Dataset: {}".format(len(train_ds)))
print("Length of Test Dataset: {}".format(len(test_ds)))
print("Shape of images as tensors: {}".format(train_ds[i]['image'].shape))
print("Label of image i: {}".format(train_ds[i]['label']))

Length of Training Dataset: 46
Length of Test Dataset: 46
Shape of images as tensors: torch.Size([3, 256, 256])
Label of image i: 3.73


In [ ]:
#Creating Dataloaders
train_dl = DataLoader(train_ds, batch_size = 46, shuffle = True)
test_dl = DataLoader(test_ds, batch_size = 46, shuffle = True)

#Iterate over batches
#for i_batch, sample_batched in enumerate(test_dl):
#  print(sample_batched['image'].shape)
#  print(sample_batched['label'])
#  break

**My Encoding**

In [ ]:
'''Hyperparameters configuration'''
#Convolutional layers
FSIZES = [1,2,3,4,5,6,7]
NFILTERS = [2,4,8,16,32]

#Pooling layers
PSIZES = [2,3,4,5]
PTYPE = ['max', 'avg']

#Fully connected layers
NEURONS = [4,8,16,32,64,128]
random.seed(5)

class Encoding:
    def __init__(self, minC, maxC, minF, maxF):
        self.n_conv = random.randint(minC, maxC)
        self.n_full = random.randint(minF, maxF)


        '''First level encoding'''
        self.first_level = []

        #Feature extraction part
        for i in range(self.n_conv):
            layer = {'type' : 'conv',
                     'nfilters' : random.choice(NFILTERS),
                     'fsize' : random.choice(FSIZES),
                     'pool' : random.choice(['max', 'avg', 'off']),
                     'psize' : random.choice(PSIZES)
                    }
            self.first_level.append(layer)

        #Fully connected part
        for i in range(self.n_full):
            layer = {'type' : 'fc',
                     'neurons' : random.choice(NEURONS)}

            self.first_level.append(layer)


        '''Second level encoding'''
        self.second_level = []
        prev = -1
        for i in range(self.n_conv):
            if prev < 1:
                prev += 1
            if prev >= 1:
                for _ in range(prev-1):
                    self.second_level.append(random.choice([0,1]))
                prev += 1

e = Encoding(2,8,1,4)

In [ ]:
def conv_out_size(W, K):
    return W - K + 3

def pool_out_size(W, K):
    return math.floor((W - K)/2) + 1

In [ ]:
def decoding(encoding):
  n_conv = encoding.n_conv
  n_full = encoding.n_full
  first_level = encoding.first_level
  second_level = encoding.second_level

  features = []
  classifier = []
  in_channels = 3 #Cambiar en caso de imagenes de 1 canal o 3 canales
  out_size = 256
  prev = -1
  pos = 0
  o_sizes = []
  for i in range(n_conv):
    layer = first_level[i]
    n_filters = layer['nfilters']
    f_size = layer['fsize']
    pad = 1
    if f_size > out_size:
        f_size = out_size - 1
    if i == 0 or i == 1:
      if layer['pool'] == 'off':
        operation = [nn.Conv2d(in_channels = in_channels, out_channels = n_filters, kernel_size = f_size, padding = pad),
                    nn.BatchNorm2d(n_filters),
                    nn.ReLU(inplace = True)]
        in_channels = n_filters
        out_size = conv_out_size(out_size, f_size)
        o_sizes.append([out_size, in_channels])

      if layer['pool'] == 'avg':
          p_size = layer['psize']
          if p_size > out_size:
              p_size = out_size - 1
          operation = [nn.Conv2d(in_channels = in_channels, out_channels = n_filters, kernel_size = f_size, padding = pad),
                      nn.BatchNorm2d(n_filters),
                      nn.ReLU(inplace = True),
                      nn.AvgPool2d(kernel_size = p_size, stride = 2)]
          in_channels = n_filters
          out_size = conv_out_size(out_size, f_size)
          out_size = pool_out_size(out_size, p_size)
          o_sizes.append([out_size, in_channels])

      if layer['pool'] == 'max':
          p_size = layer['psize']
          if p_size > out_size:
              p_size = out_size - 1
          operation = [nn.Conv2d(in_channels = in_channels, out_channels = n_filters, kernel_size = f_size, padding = pad),
                      nn.BatchNorm2d(n_filters),
                      nn.ReLU(inplace = True),
                      nn.MaxPool2d(kernel_size = p_size, stride = 2)]
          in_channels = n_filters
          out_size = conv_out_size(out_size, f_size)
          out_size = pool_out_size(out_size, p_size)
          o_sizes.append([out_size, in_channels])
    else:
      connections = second_level[pos:pos+prev]
      for c in range(len(connections)):
        if connections[c] == 1:
          in_channels += o_sizes[c][1]

      if layer['pool'] == 'off':
        operation = [nn.Conv2d(in_channels = in_channels, out_channels = n_filters, kernel_size = f_size, padding = pad),
                    nn.BatchNorm2d(n_filters),
                    nn.ReLU(inplace = True)]
        in_channels = n_filters
        out_size = conv_out_size(out_size, f_size)
        o_sizes.append([out_size, in_channels])

      if layer['pool'] == 'avg':
          p_size = layer['psize']
          if p_size > out_size:
              p_size = out_size - 1
          operation = [nn.Conv2d(in_channels = in_channels, out_channels = n_filters, kernel_size = f_size, padding = pad),
                      nn.BatchNorm2d(n_filters),
                      nn.ReLU(inplace = True),
                      nn.AvgPool2d(kernel_size = p_size, stride = 2)]
          in_channels = n_filters
          out_size = conv_out_size(out_size, f_size)
          out_size = pool_out_size(out_size, p_size)
          o_sizes.append([out_size, in_channels])

      if layer['pool'] == 'max':
          p_size = layer['psize']
          if p_size > out_size:
              p_size = out_size - 1
          operation = [nn.Conv2d(in_channels = in_channels, out_channels = n_filters, kernel_size = f_size, padding = pad),
                      nn.BatchNorm2d(n_filters),
                      nn.ReLU(inplace = True),
                      nn.MaxPool2d(kernel_size = p_size, stride = 2)]
          in_channels = n_filters
          out_size = conv_out_size(out_size, f_size)
          out_size = pool_out_size(out_size, p_size)
          o_sizes.append([out_size, in_channels])

      pos += prev
    prev += 1

    features.append(operation)
  in_size = out_size*out_size*in_channels
  for i in range(n_conv,(n_conv + n_full)):
    layer = first_level[i]
    n_neurons = layer['neurons']
    classifier += [nn.Linear(in_size, n_neurons)]
    classifier += [nn.ReLU(inplace = True)]
    in_size = n_neurons

  ##Last layer generates the numerical output for regression
  classifier += [nn.Linear(n_neurons, 1)]

  return features, classifier, o_sizes

In [ ]:
'''Networks class'''
class CNN(nn.Module):
  def __init__(self, encoding, features, classifier, sizes, init_weights = True):
    super(CNN, self).__init__()
    extraction = []
    for layer in features:
      extraction += layer
    self.extraction = nn.Sequential(*extraction)
    self.classifier = nn.Sequential(*classifier)
    self.features = features
    self.second_level = encoding.second_level
    self.sizes = sizes

  def forward(self, x):
    '''Feature extraction'''
    prev = -1
    pos = 0
    outputs = {}
    features = self.features
    #print(x.shape)
    for i in range(len(features)):
      #print('Layer: ', i)
      if i == 0 or i == 1:
        x = nn.Sequential(*features[i])(x)
        outputs[i] = x
        #print(x.shape)

      else:
        connections = self.second_level[pos:pos+prev]
        for c in range(len(connections)):
          if connections[c] == 1:
            skip_size = self.sizes[c][0] #Size comming from previous layer
            req_size = x.shape[2] #Current feature map size
            #print('X: ',x.shape)
            if skip_size > req_size:
              psize = skip_size - req_size + 1
              pool = nn.MaxPool2d(kernel_size = psize, stride = 1) #Applying pooling to adjust sizes
              x2 = pool(outputs[c])
            if skip_size == req_size:
              x2 = outputs[c]
            if req_size == skip_size + 1:
              pool = nn.MaxPool2d(kernel_size = 2, stride = 1, padding = (1,1))
              x2 = pool(outputs[c])
            if req_size == skip_size + 2:
              pad = int((req_size - skip_size)/2)
              padding = nn.ZeroPad2d(pad)
              x2 = padding(outputs[c])
            #print('X2: ',x2.shape)
            x = torch.cat((x, x2), axis = 1)

        x = nn.Sequential(*features[i])(x)
        #print('Out size: ', x.shape)
        outputs[i] = x
        pos += prev

      prev += 1


    x = torch.flatten(x,1)

    x = self.classifier(x)
    #print(x.shape)
    return x #Returning numerical regression output

**Operators**

In [ ]:
from copy import deepcopy

def crossover(x, y):
    x = deepcopy(x)
    y = deepcopy(y)

    '''First parent'''
    x_nconv = x.n_conv
    x_nfull = x.n_full
    xblocks = x.first_level
    xbinary = x.second_level

    '''Second parent'''
    y_nconv = y.n_conv
    y_nfull = y.n_full
    yblocks = y.first_level
    ybinary = y.second_level

    '''Convolutional part crossover'''
    if x_nconv > y_nconv:
        k = math.floor(y_nconv/2)
        index = list(range(x_nconv))

        '''Exchanging the last k blocks of the smaller parent'''
        for i in range(k, y_nconv):
            block = yblocks[i] #ith block
            ix = random.choice(index) #Selecting random index from larger parent
            index.remove(ix)

            #Exchange of blocks
            yblocks[i] = xblocks[ix]
            xblocks[ix] = block

    if y_nconv > x_nconv:
        k = math.floor(x_nconv/2)
        index = list(range(y_nconv))

        '''Exchanging the last k blocks of the smaller parent'''
        for i in range(k, x_nconv):
            block = xblocks[i] #ith block
            ix = random.choice(index) #Selecting random index from larger parent
            index.remove(ix)

            #Exchange of blocks
            xblocks[i] = yblocks[ix]
            yblocks[ix] = block

    if x_nconv == y_nconv:
        k = math.floor(x_nconv/2)
        index = list(range(x_nconv))

        x_part = xblocks[k:x_nconv]

        '''Exchaning last half of the blocks'''
        xblocks[k:x_nconv] = yblocks[k:y_nconv]
        yblocks[k:y_nconv] = x_part

    '''Fully-connected part'''
    if x_nfull > y_nfull:
        k = math.floor(y_nfull/2)
        index = list(range(x_nconv, x_nconv + x_nfull))

        '''Exchanging the last k blocks of the smaller parent'''
        for i in range(y_nconv + k, y_nconv + y_nfull):
            block = yblocks[i] #ith block
            ix = random.choice(index) #Selecting random index from larger parent
            index.remove(ix)

            #Exchange of blocks
            yblocks[i] = xblocks[ix]
            xblocks[ix] = block

    if y_nfull > x_nfull:
        k = math.floor(x_nfull/2)
        index = list(range(y_nconv, y_nconv + y_nfull))

        '''Exchanging the last k blocks of the smaller parent'''
        for i in range(x_nconv + k, x_nconv + x_nfull):
            block = xblocks[i] #ith block
            ix = random.choice(index) #Selecting random index from larger parent
            index.remove(ix)

            #Exchange of blocks
            xblocks[i] = yblocks[ix]
            yblocks[ix] = block

    if x_nfull == y_nfull:
        k = math.floor(x_nfull/2)

        x_part = xblocks[x_nconv + k:x_nconv + x_nfull]
        '''Exchaning last half of the blocks'''
        xblocks[x_nconv + k:x_nconv + x_nfull] = yblocks[y_nconv + k:y_nconv + y_nfull]
        yblocks[y_nconv + k:y_nconv + y_nfull] = x_part

    '''Second level'''
    if len(xbinary) > len(ybinary):
        if len(ybinary) > 1 :
            k = random.choice(list(range(1, len(ybinary))))
            partition = ybinary[k:]
            nbits = len(partition)

            if random.uniform(0,1) >= 0.5:
                ybinary[k:] = xbinary[len(xbinary) - nbits:len(xbinary)]
                xbinary[len(xbinary) - nbits:len(xbinary)] = partition
            else:
                ybinary[k:] = xbinary[:nbits]
                xbinary[:nbits] = partition

    if len(ybinary) > len(xbinary):
        if len(xbinary) > 1 :
            k = random.choice(list(range(len(xbinary))))
            partition = xbinary[k:]
            nbits = len(partition)

            if random.uniform(0,1) >= 0.5:
                xbinary[k:] = ybinary[len(ybinary) - nbits:len(ybinary)]
                ybinary[len(ybinary) - nbits:len(ybinary)] = partition
            else:
                xbinary[k:] = ybinary[:nbits]
                ybinary[:nbits] = partition

    if len(xbinary) == len(ybinary):
        if len(xbinary) > 1 :
            k = random.choice(list(range(len(xbinary))))
            partition = xbinary[k:]

            xbinary[k:] = ybinary[k:]
            ybinary[k:] = partition

    return x, y

def mutation(x):
    if random.uniform(0,1) < 0.5:
        '''Adding a new block'''
        if random.uniform(0,1) > 0.5:
            #Adding a fully-connected block
            layer = {'type' : 'fc',
                     'neurons' : random.choice(NEURONS)}

            #Choosing a random index to insert the new block
            index = list(range(x.n_conv, x.n_conv + x.n_full))
            ix = random.choice(index)

            x.first_level.insert(ix, layer)
            x.n_full += 1

        else:
            #Adding a convolutional block
            layer = {'type' : 'conv',
                     'nfilters' : random.choice(NFILTERS),
                     'fsize' : random.choice(FSIZES),
                     'pool' : random.choice(['max', 'avg', 'off']),
                     'psize' : random.choice(PSIZES)
                    }
            #Choosing a random index to insert the new block
            index = list(range(x.n_conv))
            ix = random.choice(index)

            x.first_level.insert(ix, layer)
            x.n_conv += 1

            if ix > 1:
                new_bits = []
                for i in range(ix - 1):
                    new_bits.append(random.choice([0,1]))
                pos = int(0.5*(ix**2) - 1.5*(ix) + 1)
                start = pos + len(new_bits)
                for bit in new_bits:
                    x.second_level.insert(pos, bit)
                    pos += 1

                rest = x.n_conv - ix - 1
                add = ix
                for j in range(rest):
                    x.second_level.insert(start+add-1, random.choice([0,1]))
                    start += add
                    ix += 1

            if ix == 0 or ix == 1:
                if x.n_conv - 1 == 2:
                    x.second_level.append(random.choice([0,1]))
                else:
                    add = 0
                    for i in range(2, x.n_conv):
                        pos = int(0.5*(ix**2) - 1.5*(ix) + 1) + add
                        x.second_level.insert(pos, random.choice([0,1]))
                        add += 1

    else:
        '''Changing hyperparameters in one block'''
        if random.uniform(0,1) > 0.5:
            '''Re-starting a fully-connected block'''
            index = list(range(x.n_conv, x.n_conv + x.n_full))
            ix = random.choice(index)
            new_layer = {'type' : 'fc',
                         'neurons' : random.choice(NEURONS)}
            #Switching fully-connected block
            x.first_level[ix] = new_layer

        else:
            '''Re-starting a convolutional block'''
            index = list(range(x.n_conv))
            ix = random.choice(index)
            new_layer = {'type' : 'conv',
                     'nfilters' : random.choice(NFILTERS),
                     'fsize' : random.choice(FSIZES),
                     'pool' : random.choice(['max', 'avg', 'off']),
                     'psize' : random.choice(PSIZES)
                    }

            #Switching convolutional block
            x.first_level[ix] = new_layer

        '''Modifying connections in second level'''
        if len(x.second_level) > 0:
            k = random.choice(list(range(len(x.second_level))))
            #Flipping one bit in the second level
            if x.second_level[k] == 1:
                x.second_level[k] = 0
            else:
                x.second_level[k] = 1


def selection(tournament, style):
    '''Stochastic tournament selection'''
    if style == 'max':
        if random.uniform(0,1) <= 0.8:
            p = max(tournament, key = lambda x: x[1])
        else:
            p = random.choice(tournament)
    else:
        if random.uniform(0,1) <= 0.8:
            p = min(tournament, key = lambda x: x[1])
        else:
            p = random.choice(tournament)

    return p

**Training**

In [ ]:
def entrenamiento(device, cnn, max_epochs, loss_func, optimizer, train_dl):
  cnn.train() #Configurando la red en modo de entrenamiento
  '''Iterando sobre las épocas de entrenamiento (Linea 1)'''
  torch.set_default_tensor_type('torch.FloatTensor')
  #torch.set_default_tensor_type(device)
  for epoch in range(max_epochs):

    '''Iterando sobre los batches (si el tamaño de batch es de 100, solo habrá un batch) (Linea 2)'''
    for i, data in enumerate(train_dl, 0):

      '''Leyendo datos del batch (Linea 3 y Linea 4)'''
      xb, yb = data['image'], data['label'] #Leyendo entrada y etiqueta
      xb = xb.type(torch.float32).to(device, dtype = torch.float32) #Moviendo entrada a GPU
      yb = yb.to(device, dtype = torch.float32) #Moviendo etiqueta a GPU
      '''Obteniendo salida de la CNN (Linea 5)'''
      y_pred = cnn(xb)

      '''Obteniendo valor de la función de costo (RMSE) (Linea 6)'''
      loss = loss_func(torch.transpose(y_pred, 0, 1), yb.unsqueeze(0))

      '''Obteniendo gradiente y hacer paso hacia atras (Linea 7)'''
      loss.backward() #Paso hacia atrás (derivadas)
      optimizer.step() #Ajustando pasos con optimizador
      optimizer.zero_grad(set_to_none = True) #Reiniciando gradientes a 0

  return loss.item(), cnn #Devuelve la perdida final y la cnn entrenada

def validacion(device, cnn, test_dl):
  '''Inicializar perdida (Linea 1)'''
  cnn.eval() #Configurando la red en modo evaluacion

  '''Iterando sobre los batches (si el tamaño de batch es de 100, solo habrá un batch) (Linea 2)'''
  for i, data in enumerate(test_dl, 0):
    '''Leyendo datos del batch (Linea 3 y Linea 4)'''
    xb, yb = data['image'], data['label'] #Leyendo entrada y etiqueta
    xb = xb.type(torch.float32).to(device, dtype = torch.float32) #Moviendo entrada a GPU
    yb = yb.to(device, dtype = torch.float32) #Moviendo etiqueta a GPU
    '''Obteniendo salida de la CNN (Linea 5)'''
    y_pred = cnn(xb).detach().cpu().numpy().reshape(46,)
    y = yb.detach().cpu().numpy()
    print('real  {0} y estimado {1}'.format(y, y_pred))

  SMAPE = np.mean(np.abs((y_pred+1) - (y + 1))/(np.abs(y_pred+1) + np.abs(y+1)))
  MAPE = np.mean(np.abs((y_pred+1) - (y + 1))/(y+1))
  return SMAPE, MAPE

In [ ]:
def training(device, model, n_epochs, loss_func, train_dl, test_dl, lr, w, max_params):
    #Number of parameters
    params = sum(p.numel() for p in model.parameters() if p.requires_grad)

    model.to(device)

    #Optimizer
    opt = optim.Adam(model.parameters(), lr = lr)

    #Obtaining training accuracy
    mse, cnn = entrenamiento(device, model, n_epochs, loss_func, opt, train_dl)

    SMAPE, MAPE = validacion(device, cnn, test_dl)

    #Fitness function based on accuracy and No. of parameters
    #f = (1 - w)*(MAPE) - w*((max_params - params)/max_params)

    #Append results to multiprocessing list
    return MAPE, SMAPE, params, mse


**DeepGA**

In [ ]:
#Maximun and minimum numbers of layers to initialize networks
min_conv = 2
max_conv = 6
min_full = 1
max_full = 4

'''Genetic Algorithm Parameters'''
cr = 0.7 #Crossover rate
mr = 0.5 #Mutation rate
N = 20 #Population size
T = 50 #Number of generations
t_size = 5 #tournament size
w = 0.1 #penalization weight
max_params = 3e6
num_epochs =  30
lr = 1e-3

#Defining loss function
loss_func = torch.nn.MSELoss()

In [ ]:
import timeit
import pickle
import pandas as pd

'''Initialize population'''
print('Initialize population')
start = timeit.default_timer()
pop = []
bestAcc = []
bestF = []
bestParams = []
bestRMSE = []
while len(pop) < N:

    #Creating genomes (genetic encoding)
    e1 = Encoding(min_conv,max_conv,min_full,max_full)

    #Decoding the networks
    network1 = decoding(e1)

    #Creating the CNNs
    cnn1 = CNN(e1, network1[0], network1[1], network1[2])


    fitness, smape, params, mse =  training(device, cnn1, num_epochs, loss_func, train_dl, test_dl, lr, w, max_params)
    pop.append([e1, fitness, smape, params, mse])

'''Genetic Algorithm'''
for t in range(T):
    print('Generation: ', t)

    #Parents Selection
    parents = []
    while len(parents) < int(N/2):
        #Tournament Selection
        tournament = random.sample(pop, t_size)
        p1 = selection(tournament, 'min')
        tournament = random.sample(pop, t_size)
        p2 = selection(tournament, 'min')
        while p1 == p2:
            tournament = random.sample(pop, t_size)
            p2 = selection(tournament, 'min')

        parents.append(p1)
        parents.append(p2)

    #Reproduction
    offspring = []
    while len(offspring) < int(N/2):
        par = random.sample(parents, 2)
        #Crossover + Mutation
        if cr >= random.uniform(0,1): #Crossover
            p1 = par[0][0]
            p2 = par[1][0]
            c1, c2 = crossover(p1, p2)

            #Mutation
            if mr >= random.uniform(0,1):
                mutation(c1)

            if mr >= random.uniform(0,1):
                mutation(c2)


            #Decoding the network
            network1 = decoding(c1)
            network2 = decoding(c2)

            #Creating the CNN
            cnn1 = CNN(c1, network1[0], network1[1], network1[2])
            cnn2 = CNN(c2, network2[0], network2[1], network2[2])

            #Evaluate individuals
            fitness, smape, params, mse =  training(device, cnn1, num_epochs, loss_func, train_dl, test_dl, lr, w, max_params)
            offspring.append([e1, fitness, smape, params, mse])

            fitness, smape, params, mse =  training(device, cnn2, num_epochs, loss_func, train_dl, test_dl, lr, w, max_params)
            offspring.append([e1, fitness, smape, params, mse])


    #Replacement with elitism
    pop = pop + offspring
    pop.sort(key = lambda x: x[1])
    pop = pop[:N]

    leader = min(pop, key = lambda x: x[1])
    bestAcc.append(leader[2])
    bestF.append(leader[1])
    bestParams.append(leader[3])
    bestRMSE.append(leader[4])


    print('Best fitness (MAPE): ', leader[1])
    print('Best SMAPE: ', leader[2])
    print('Best No. of Params: ', leader[3])
    print('Best training MSE: ', leader[4])
    print('No. of Conv. Layers: ', leader[0].n_conv)
    print('No. of FC Layers: ', leader[0].n_full)
    print('--------------------------------------------')

results = pd.DataFrame(list(zip(bestAcc, bestF, bestParams, bestRMSE)), columns = ['SMAPE', 'Fitness (MAPE)', 'No. Params', 'MSE'])
final_networks = []
final_connections = []
objects = []
for member in pop:
    p = member[0]
    objects.append(p)
    n_conv = p.n_conv
    n_full = p.n_full
    description = 'The network has ' + str(n_conv) + ' convolutional layers ' + 'with: '
    for i in range(n_conv):
        nfilters = str(p.first_level[i]['nfilters'])
        fsize = str(p.first_level[i]['fsize'])
        pool = str(p.first_level[i]['pool'])
        psize = str(p.first_level[i]['psize'])
        layer = '(' + nfilters + ', ' + fsize + ', ' + pool + ', ' + psize + ') '
        description += layer
    description += 'and '
    description += str(n_full)
    description += ' '
    description += 'fully-connected layers with: '
    for i in range(n_conv, n_conv+n_full):
        neurons = str(p.first_level[i]['neurons'])
        layer = '(' + neurons + ')'
        description += layer
    description += ' neurons'
    final_networks.append(description)

    connections = ''
    for bit in p.second_level:
        if bit == 1:
            connections += 'one - '
        if bit == 0:
            connections += 'zero - '
    final_connections.append(connections)


final_population = pd.DataFrame(list(zip(final_networks, final_connections)), columns = ['Network Architecture', 'Connections'])

'''Saving Results as CSV'''
final_population.to_excel('final_CNN_population50ECIE3H2D.xls', index = False)
results.to_excel('log_Neuroevolution50ECIE3H2D.xls', index = False)
stop = timeit.default_timer()
execution_time = (stop-start)/3600
print("Execution time: ", execution_time)

#Saving objects
with open('cnns50ECIE3H2D.pkl', 'wb') as output:
    pickle.dump(objects, output, pickle.HIGHEST_PROTOCOL)
    output.close()


Initialize population


/usr/local/lib/python3.10/dist-packages/torch/__init__.py:614: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at ../torch/csrc/tensor/python_tensor.cpp:451.)
  _C._set_default_tensor_type(t)


Streaming output truncated to the last 5000 lines.
 0.95 0.61 4.15 3.6  8.38 3.45 6.1  0.   0.   3.7  2.16 0.   2.72 0.
 0.02 0.01 2.53 0.  ] y estimado [0.09252748 0.09252748 0.09252748 0.09252748 0.09252748 0.09252748
 0.09252748 0.09252748 0.09252748 0.09252748 0.09252748 0.09252748
 0.09252748 0.09252748 0.09252748 0.09252748 0.09252748 0.09252748
 0.09252748 0.09252748 0.09252748 0.09252748 0.09252748 0.09252748
 0.09252748 0.09252748 0.09252748 0.09252748 0.09252748 0.09252748
 0.09252748 0.09252748 0.09252748 0.09252748 0.09252748 0.09252748
 0.09252748 0.09252748 0.09252748 0.09252748 0.09252748 0.09252748
 0.09252748 0.09252748 0.09252748 0.09252748]
real  [0.02 1.5  2.53 0.96 3.27 0.   0.   4.15 0.02 0.01 0.02 0.95 0.   2.24
 2.44 3.45 0.   0.   0.01 3.73 0.   3.6  5.63 0.   3.7  0.   8.38 0.
 0.   0.   2.16 2.72 2.36 0.   0.65 0.   2.89 0.   4.5  0.   0.   0.
 6.1  0.61 0.82 0.  ] y estimado [0.21874504 0.22507575 0.22410874 0.22110851 0.22709838 0.19967875
 0.20609227 0.227

<ipython-input-21-3060c6e2f8c8>:141: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  final_population.to_excel('final_CNN_population50ECIE3H2D.xls', index = False)
<ipython-input-21-3060c6e2f8c8>:142: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be gl

In [ ]:
!cp final_CNN_population50ECIE3H2D.xls /content/gdrive/MyDrive/DB46HSILAB/OUT_HSI_3H_2D
!cp log_Neuroevolution50ECIE3H2D.xls /content/gdrive/MyDrive/DB46HSILAB/OUT_HSI_3H_2D
!cp cnnsCIE50E3H2D.pkl /content/gdrive/MyDrive/DB46HSILAB/OUT_HSI_3H_2D

cp: cannot stat 'cnnsCIE50E3H2D.pkl': No such file or directory


In [ ]:
from google.colab import files
files.download('final_CNN_population50ECIE3H2D.xls')
files.download('log_Neuroevolution50ECIE3H2D.xls')
files.download('cnns50ECIE3H2D.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
bestInd = pop[0][0]

In [ ]:
import pickle

exampleObj = bestInd

fileObj = open('/content/gdrive/MyDrive/DB46HSILAB/OUT_HSI_3H_2D/bestInd.obj', 'wb')
pickle.dump(exampleObj,fileObj)
fileObj.close()

In [ ]:
!cp bestInd.obj /content/gdrive/MyDrive/DB46HSILAB/OUT_HSI_3H_2D

cp: cannot stat 'bestInd.obj': No such file or directory


In [ ]:
#cnn = decoding(e)
networkFinal = decoding(bestInd) #decoding(pop[0][0])
'''Construimos la CNN'''
modelFinal = CNN(bestInd, networkFinal[0], networkFinal[1], networkFinal[2])

In [ ]:
modelFinal

CNN(
  (extraction): Sequential(
    (0): Conv2d(3, 16, kernel_size=(7, 7), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(16, 32, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(32, 16, kernel_size=(6, 6), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU(inplace=True)
    (11): MaxPool2d(kernel_size=4, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Linear(in_features=13456, out_features=4, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(

In [ ]:
new_num_epoch = 200
fitness, smape, params, mse =  training(device, modelFinal, new_num_epoch, loss_func, train_dl, test_dl, lr, w, max_params)

real  [2.16 5.63 0.   0.   0.   0.02 2.72 0.   0.82 2.44 0.96 3.6  6.1  8.38
 0.01 2.36 3.27 0.   0.   0.   4.15 2.24 0.   1.5  3.45 0.01 0.02 4.5
 0.02 0.   0.   0.   0.95 0.   0.   0.   0.   3.73 0.65 0.   2.89 2.53
 0.   0.   3.7  0.61] y estimado [5.9261837e+00 8.3409348e+00 1.8634228e+00 3.6418512e-03 6.6867036e-01
 3.2523739e+00 6.6513906e+00 1.2337748e+00 3.0328646e+00 6.4428902e+00
 4.1794448e+00 6.9205079e+00 6.8398800e+00 1.2519197e+01 2.7275002e+00
 5.9543347e+00 7.4932227e+00 1.4251999e+00 3.2027510e-01 3.6418512e-03
 6.8507566e+00 6.0835390e+00 3.6418512e-03 5.2312050e+00 7.5610781e+00
 2.6324797e+00 1.1810061e+00 6.5029206e+00 1.7527312e+00 2.2787588e+00
 3.6418512e-03 1.3103746e+00 4.4704595e+00 3.6418512e-03 3.6418512e-03
 1.9341617e+00 3.6418512e-03 6.5794425e+00 2.9645195e+00 3.3345804e+00
 5.8955240e+00 6.1373005e+00 2.8455977e+00 5.5300194e-01 7.0305767e+00
 2.6538377e+00]


In [ ]:
fitness

1.1123145

In [ ]:
100-(fitness*100)

-11.231446266174316

In [ ]:
smape

0.30927488

In [ ]:
params

77237

In [ ]:
mse

0.02256053499877453

In [ ]:
import pickle

fileObj = open('/content/gdrive/MyDrive/DB46HSILAB/OUT_HSI_3H_2D/30 epocas/bestInd.obj', 'rb')
bestInd = pickle.load(fileObj)
fileObj.close()
print(bestInd)

In [ ]:
networkFinalDrive = decoding(bestInd) #decoding(pop[0][0])
'''Construimos la CNN'''
modelFinalDrive = CNN(e, networkFinal[0], networkFinal[1], networkFinal[2])
modelFinalDrive